### Data Preprocessing
Import the necessary packages, read the csv file, connect to the database, and store data from csv file to a relational database table for sql querying

In [1]:
import pandas as pd
import sqlite3

df = pd.read_csv("Sample - Superstore.csv", encoding='latin-1')
conn = sqlite3.connect("superstore.db")  

# Write DataFrame to a SQL table
df.to_sql("superstore", conn, index=False, if_exists="replace")

9994

In [2]:
query = """ 
SELECT * 
FROM superstore
LIMIT 5;
"""

table = pd.read_sql_query(query, conn)
print(table)

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

### Task 1: Analyze sales and profit trends by region, product category, and customer segment

In [4]:
# Total sales and profit by region
query = """
SELECT Region, ROUND(SUM(Sales), 2) AS Total_Sales, ROUND(SUM(Profit), 2) AS Total_Profit
FROM superstore
GROUP BY Region
ORDER BY Total_Profit DESC;
"""
region_sales = pd.read_sql_query(query, conn)
print('Total Sales and Profit by Region:')
print(region_sales)

# Total sales and profit by category
query = """
SELECT Category, ROUND(SUM(Sales), 2) AS Total_Sales, ROUND(SUM(Profit), 2) AS Total_Profit
FROM superstore
GROUP BY Category
ORDER BY Total_Profit DESC;
"""

category_sales = pd.read_sql_query(query, conn)
print('\nTotal Sales and Profit by Category:')
print(category_sales)

# Total sales and profit by segment
query = """
SELECT Segment, ROUND(SUM(Sales), 2) AS Total_Sales, ROUND(SUM(Profit), 2) AS Total_Profit
FROM superstore
GROUP BY Segment
ORDER BY Total_Profit DESC;
"""

segment_sales = pd.read_sql_query(query, conn)
print('\nTotal Sales and Profit by Segment:')
print(segment_sales)


Total Sales and Profit by Region:
    Region  Total_Sales  Total_Profit
0     West    725457.82     108418.45
1     East    678781.24      91522.78
2    South    391721.91      46749.43
3  Central    501239.89      39706.36

Total Sales and Profit by Category:
          Category  Total_Sales  Total_Profit
0       Technology    836154.03     145454.95
1  Office Supplies    719047.03     122490.80
2        Furniture    741999.80      18451.27

Total Sales and Profit by Segment:
       Segment  Total_Sales  Total_Profit
0     Consumer   1161401.34     134119.21
1    Corporate    706146.37      91979.13
2  Home Office    429653.15      60298.68


### Task 2: Calculate KPIs such as average discount and profit margin.

In [5]:
# Average discount and profit margin calculations for superstore
query_metrics = """
SELECT
  ROUND(AVG(Discount), 4) AS Avg_Discount,
  ROUND(SUM(Discount * Sales) / SUM(Sales), 4) AS Weighted_Avg_Discount,
  ROUND(SUM(Profit) / SUM(Sales), 4) AS Profit_Margin
FROM superstore;
"""
metrics = pd.read_sql_query(query_metrics, conn)
print('Average Discount and Profit Margin:')
print(metrics)

# Average discount and profit margin by category
query_category = """
SELECT
  Category,
  ROUND(AVG(Discount), 4) AS Avg_Discount,
  ROUND(SUM(Discount * Sales) / SUM(Sales), 4) AS Weighted_Avg_Discount,
  ROUND(SUM(Profit) / SUM(Sales), 4) AS Profit_Margin
FROM superstore
GROUP BY Category
ORDER BY Profit_Margin DESC;
"""

category = pd.read_sql_query(query_category, conn)
print('\nAverage Discount and Profit Margin by Category:')
print(category)

# Average discount and profit margin by region
query_region = """
SELECT
  Region,
  ROUND(AVG(Discount), 4) AS Avg_Discount,
  ROUND(SUM(Discount * Sales) / SUM(Sales), 4) AS Weighted_Avg_Discount,
  ROUND(SUM(Profit) / SUM(Sales), 4) AS Profit_Margin
  FROM superstore
  GROUP BY Region
  ORDER BY Profit_Margin DESC;
"""

region = pd.read_sql_query(query_region, conn)
print('\nAverage Discount and Profit Margin by Region:')
print(region)

Average Discount and Profit Margin:
   Avg_Discount  Weighted_Avg_Discount  Profit_Margin
0        0.1562                 0.1404         0.1247

Average Discount and Profit Margin by Category:
          Category  Avg_Discount  Weighted_Avg_Discount  Profit_Margin
0       Technology        0.1323                 0.1467         0.1740
1  Office Supplies        0.1573                 0.1063         0.1704
2        Furniture        0.1739                 0.1665         0.0249

Average Discount and Profit Margin by Region:
    Region  Avg_Discount  Weighted_Avg_Discount  Profit_Margin
0     West        0.1093                 0.1288         0.1494
1     East        0.1454                 0.1394         0.1348
2    South        0.1473                 0.1449         0.1193
3  Central        0.2404                 0.1552         0.0792


### Task 3: Identify top-performing products and underperforming categories to provide actionable business insights.

In [6]:
# Top 10 performing products based on sales and profit
query_top_products = """
SELECT
    "Product Name", Category,
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit
    FROM superstore
    GROUP BY "Product Name"
    ORDER BY Total_Profit DESC
    LIMIT 10;
"""

top_products = pd.read_sql_query(query_top_products, conn)
print('Top 10 Performing Products:')
print(top_products)

Top 10 Performing Products:
                                        Product Name         Category  \
0              Canon imageCLASS 2200 Advanced Copier       Technology   
1  Fellowes PB500 Electric Punch Plastic Comb Bin...  Office Supplies   
2               Hewlett Packard LaserJet 3310 Copier       Technology   
3                 Canon PC1060 Personal Laser Copier       Technology   
4  HP Designjet T520 Inkjet Large Format Printer ...       Technology   
5                  Ativa V4110MDD Micro-Cut Shredder       Technology   
6   3D Systems Cube Printer, 2nd Generation, Magenta       Technology   
7  Plantronics Savi W720 Multi-Device Wireless He...       Technology   
8               Ibico EPK-21 Electric Binding System  Office Supplies   
9                  Zebra ZM400 Thermal Label Printer       Technology   

   Total_Sales  Total_Profit  
0     61599.82      25199.93  
1     27453.38       7753.04  
2     18839.69       6983.88  
3     11619.83       4570.93  
4     18374.9

### Data Visualization: Interactive Plots

Let's visualize our key findings using interactive plots. These visualizations help identify patterns and insights more effectively.

In [7]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Regional Performance: Sales and Profit
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(name="Sales", x=region_sales["Region"], y=region_sales["Total_Sales"], marker_color='lightblue'),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(name="Profit", x=region_sales["Region"], y=region_sales["Total_Profit"], marker_color='darkgreen'),
    secondary_y=True,
)

fig.update_layout(
    title="Sales and Profit by Region",
    xaxis_title="Region",
    yaxis_title="Total Sales ($)",
    yaxis2_title="Total Profit ($)",
    barmode='group'
)

fig.show()

# 2. Category Performance
fig = px.bar(category_sales, 
             x="Category", 
             y=["Total_Sales", "Total_Profit"],
             title="Sales and Profit by Category",
             barmode="group",
             color_discrete_map={"Total_Sales": "lightblue", "Total_Profit": "darkgreen"})

fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Amount ($)",
    legend_title="Metric"
)

fig.show()

# 3. Discount Impact Analysis
query_discount = """
SELECT 
    ROUND(Discount, 2) as Discount_Rate,
    ROUND(SUM(Sales), 2) as Total_Sales,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND(SUM(Profit)/SUM(Sales), 4) as Profit_Margin
FROM superstore
GROUP BY ROUND(Discount, 2)
HAVING Total_Sales > 0
ORDER BY Discount_Rate;
"""
discount_analysis = pd.read_sql_query(query_discount, conn)

fig = px.scatter(discount_analysis, 
                x="Discount_Rate", 
                y="Profit_Margin",
                size="Total_Sales",  
                color="Total_Profit",  
                title="Discount Rate vs. Profit Margin",
                labels={
                    "Discount_Rate": "Discount Rate",
                    "Profit_Margin": "Profit Margin",
                    "Total_Sales": "Total Sales",
                    "Total_Profit": "Total Profit"
                })

fig.update_layout(
    xaxis_title="Discount Rate",
    yaxis_title="Profit Margin",
    coloraxis_colorbar_title="Total Profit ($)"
)

fig.show()